In [1]:
!python --version

Python 3.7.13


# <span style="font-family:AppleGothic; font-weight:bolder"><a href="https://dacon.io/competitions/official/235901/overview/description">DACON-중고차 가격 예측 경진대회</a></span>

In [2]:
# 라이브러리 불러오기
import os
import platform

import warnings
warnings.filterwarnings(action = "ignore")

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import missingno as msno
import seaborn as sns
%matplotlib inline

In [3]:
# 

ModuleNotFoundError: No module named 'pycaret'

In [5]:
import pycaret

ModuleNotFoundError: No module named 'pycaret'

In [8]:
!cd

C:\Users\User\OneDrive\바탕 화면\DACON-Competition\[Basic] 중고차 가격 예측 경진대회
